In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

VER = 152
USE = 'clicks'

PART = 'B'

In [ ]:
import pandas as pd, cudf, numpy as np, gc
pd.set_option('display.max_columns', 500)
import matplotlib.pyplot as plt

In [ ]:
USR = np.load(f'test_user_{PART}.npy')
len( USR )

In [ ]:
LOAD = 709
OFFSET = ''

In [ ]:
users = cudf.read_parquet(f'../../data/train_with_features/train_v{VER}_{PART}.pqt')
print( users.shape )
users.head()

# Add Weights

In [ ]:
for k in [1,3,4,5,6,7]:

    dfA = cudf.read_csv(f'../../data/candidate_scores/submission_v564_v{k}.csv')
    dfA['user'] = dfA.session_type.str.partition('_')[0].astype('int32')
    dfA = dfA.drop('session_type',axis=1)

    tmpA = dfA.labels.str.split(' ').explode().astype('int32').rename('item').to_frame()
    tmpA['pos'] = np.tile( np.arange(50),tmpA.shape[0]//50 ).astype('int32')

    dfA = dfA[['user']]
    dfA = dfA.merge(tmpA, left_index=True, right_index=True)
    del tmpA

    dfB = cudf.read_csv(f'../../data/candidate_scores/submission_v564_wgt_v{k}.csv')
    dfB['user'] = dfB.session_type.str.partition('_')[0].astype('int32')
    dfB = dfB.drop('session_type',axis=1)

    tmpB = dfB.labels.str.split(' ').explode().astype('float32').rename('item').to_frame()
    tmpB['pos'] = np.tile( np.arange(50),tmpB.shape[0]//50 ).astype('int32')

    dfB = dfB[['user']]
    dfB = dfB.merge(tmpB, left_index=True, right_index=True)
    dfB.columns = ['user',f'score_{k}','pos']
    del tmpB

    dfA = dfA.merge(dfB, on=['user','pos'], how='left').drop('pos',axis=1)
    dfA = dfA.loc[dfA[f'score_{k}']!=-2]
    del dfB
      
    #dfA = dfA.rename({'pos':f'pos_{k}'},axis=1)
    users = users.merge(dfA,on=['user','item'],how='left').fillna(-3)
    del dfA
    
    gc.collect()
    
    print(k,users.shape)

In [ ]:
#import gc
#users = users.to_pandas()
#gc.collect()

In [ ]:
USE2 = 'orders'
OFFSET = 10
for j,k in enumerate(['../../data/candidate_scores/submission_v564_F',
                      '../../data/candidate_scores/submission_v612']):

    dfA = cudf.read_csv(f'{k}.csv')
    dfA = dfA.loc[dfA.session_type.str.contains(USE2)]
    dfA['user'] = dfA.session_type.str.partition('_')[0].astype('int32')
    dfA = dfA.drop('session_type',axis=1)

    tmpA = dfA.labels.str.split(' ').explode().astype('int32').rename('item').to_frame()
    tmpA['pos'] = np.tile( np.arange(50),tmpA.shape[0]//50 ).astype('int32')

    dfA = dfA[['user']]
    dfA = dfA.merge(tmpA, left_index=True, right_index=True)
    del tmpA

    dfB = cudf.read_csv(f'{k}_wgt.csv')
    dfB = dfB.loc[dfB.session_type.str.contains(USE2)]
    dfB['user'] = dfB.session_type.str.partition('_')[0].astype('int32')
    dfB = dfB.drop('session_type',axis=1)

    tmpB = dfB.labels.str.split(' ').explode().astype('float32').rename('item').to_frame()
    tmpB['pos'] = np.tile( np.arange(50),tmpB.shape[0]//50 ).astype('int32')

    dfB = dfB[['user']]
    dfB = dfB.merge(tmpB, left_index=True, right_index=True)
    dfB.columns = ['user',f'score_{j+OFFSET}','pos']
    del tmpB

    dfA = dfA.merge(dfB, on=['user','pos'], how='left').drop('pos',axis=1)
    dfA = dfA.loc[dfA[f'score_{j+OFFSET}']!=-2]
    del dfB
      
    #dfA = dfA.rename({'pos':f'pos_{k}'},axis=1)
    users = users.merge(dfA,on=['user','item'],how='left').fillna(-3)
    del dfA
    
    gc.collect()
    
    print(k,users.shape)

In [ ]:
USE2 = 'clicks'
OFFSET = 12
for j,k in enumerate(['../../data/candidate_scores/submission_v564_F',
                      '../../data/candidate_scores/submission_v612']):

    dfA = cudf.read_csv(f'{k}.csv')
    dfA = dfA.loc[dfA.session_type.str.contains(USE2)]
    dfA['user'] = dfA.session_type.str.partition('_')[0].astype('int32')
    dfA = dfA.drop('session_type',axis=1)

    tmpA = dfA.labels.str.split(' ').explode().astype('int32').rename('item').to_frame()
    tmpA['pos'] = np.tile( np.arange(50),tmpA.shape[0]//50 ).astype('int32')

    dfA = dfA[['user']]
    dfA = dfA.merge(tmpA, left_index=True, right_index=True)
    del tmpA

    dfB = cudf.read_csv(f'{k}_wgt.csv')
    dfB = dfB.loc[dfB.session_type.str.contains(USE2)]
    dfB['user'] = dfB.session_type.str.partition('_')[0].astype('int32')
    dfB = dfB.drop('session_type',axis=1)

    tmpB = dfB.labels.str.split(' ').explode().astype('float32').rename('item').to_frame()
    tmpB['pos'] = np.tile( np.arange(50),tmpB.shape[0]//50 ).astype('int32')

    dfB = dfB[['user']]
    dfB = dfB.merge(tmpB, left_index=True, right_index=True)
    dfB.columns = ['user',f'score_{j+OFFSET}','pos']
    del tmpB

    dfA = dfA.merge(dfB, on=['user','pos'], how='left').drop('pos',axis=1)
    dfA = dfA.loc[dfA[f'score_{j+OFFSET}']!=-2]
    del dfB
      
    #dfA = dfA.rename({'pos':f'pos_{k}'},axis=1)
    users = users.merge(dfA,on=['user','item'],how='left').fillna(-3)
    del dfA
    
    gc.collect()
    
    print(k,users.shape)

In [ ]:
#users = users.to_pandas()
#gc.collect()

In [ ]:
if USE=='clicks': USE2 = 'orders'
else: USE2 = 'clicks'
    
OFFSET = 30
for j,k in enumerate(['../../data/candidate_scores/submission_v709']):

    dfA = pd.read_csv(f'{k}.csv')
    dfA = dfA.loc[dfA.session_type.str.contains(USE2)]
    dfA['user'] = dfA.session_type.str.partition('_')[0].astype('int32')
    dfA = dfA.loc[dfA.user.isin(USR)].reset_index(drop=True)
    dfA = dfA.drop('session_type',axis=1)
    dfA = cudf.DataFrame(dfA)

    tmpA = dfA.labels.str.split(' ').explode().astype('int32').rename('item').to_frame()
    tmpA['pos'] = np.tile( np.arange(100),tmpA.shape[0]//100 ).astype('int32')

    dfA = dfA[['user']]
    dfA = dfA.merge(tmpA, left_index=True, right_index=True)
    del tmpA

    dfB = pd.read_csv(f'{k}_wgt.csv')
    dfB = dfB.loc[dfB.session_type.str.contains(USE2)]
    dfB['user'] = dfB.session_type.str.partition('_')[0].astype('int32')
    dfB = dfB.loc[dfB.user.isin(USR)].reset_index(drop=True)
    dfB = dfB.drop('session_type',axis=1)
    dfB = cudf.DataFrame(dfB)

    tmpB = dfB.labels.str.split(' ').explode().astype('float32').rename('item').to_frame()
    tmpB['pos'] = np.tile( np.arange(100),tmpB.shape[0]//100 ).astype('int32')

    dfB = dfB[['user']]
    dfB = dfB.merge(tmpB, left_index=True, right_index=True)
    dfB.columns = ['user',f'score_{j+OFFSET}','pos']
    del tmpB

    dfA = dfA.merge(dfB, on=['user','pos'], how='left').drop('pos',axis=1)
    dfA = dfA.loc[dfA[f'score_{j+OFFSET}']!=-2]
    del dfB
      
    #dfA = dfA.rename({'pos':f'pos_{k}'},axis=1)
    users = users.merge(dfA,on=['user','item'],how='left').fillna(-3)
    del dfA
    
    gc.collect()
    
    print(k,users.shape)

In [ ]:
USE2 = 'orders'
OFFSET = 14

for j,k in enumerate( range(20,28) ):

    dfA = cudf.read_csv(f'../../data/candidate_scores/submission_v564_v{k}.csv')
    dfA = dfA.loc[dfA.session_type.str.contains(USE2)]
    dfA['user'] = dfA.session_type.str.partition('_')[0].astype('int32')
    dfA = dfA.drop('session_type',axis=1)

    tmpA = dfA.labels.str.split(' ').explode().astype('int32').rename('item').to_frame()
    tmpA['pos'] = np.tile( np.arange(50),tmpA.shape[0]//50 ).astype('int32')

    dfA = dfA[['user']]
    dfA = dfA.merge(tmpA, left_index=True, right_index=True)
    del tmpA

    dfB = cudf.read_csv(f'../../data/candidate_scores/submission_v564_wgt_v{k}.csv')
    dfB = dfB.loc[dfB.session_type.str.contains(USE2)]
    dfB['user'] = dfB.session_type.str.partition('_')[0].astype('int32')
    dfB = dfB.drop('session_type',axis=1)

    tmpB = dfB.labels.str.split(' ').explode().astype('float32').rename('item').to_frame()
    tmpB['pos'] = np.tile( np.arange(50),tmpB.shape[0]//50 ).astype('int32')

    dfB = dfB[['user']]
    dfB = dfB.merge(tmpB, left_index=True, right_index=True)
    dfB.columns = ['user',f'score_{j+OFFSET}','pos']
    del tmpB

    dfA = dfA.merge(dfB, on=['user','pos'], how='left').drop('pos',axis=1)
    dfA = dfA.loc[dfA[f'score_{j+OFFSET}']!=-2]
    del dfB
      
    #dfA = dfA.rename({'pos':f'pos_{k}'},axis=1)
    users = users.merge(dfA,on=['user','item'],how='left').fillna(-3)
    del dfA
    
    gc.collect()
    
    print(k,users.shape)

In [ ]:
users = users.to_pandas()
gc.collect()

In [ ]:
USE2 = 'orders'
OFFSET = 22
for j,k in enumerate(['../../data/candidate_scores/submission_v614']):

    dfA = cudf.read_csv(f'{k}.csv')
    dfA = dfA.loc[dfA.session_type.str.contains(USE2)]
    dfA['user'] = dfA.session_type.str.partition('_')[0].astype('int32')
    dfA = dfA.drop('session_type',axis=1)

    tmpA = dfA.labels.str.split(' ').explode().astype('int32').rename('item').to_frame()
    tmpA['pos'] = np.tile( np.arange(50),tmpA.shape[0]//50 ).astype('int32')

    dfA = dfA[['user']]
    dfA = dfA.merge(tmpA, left_index=True, right_index=True)
    del tmpA

    dfB = cudf.read_csv(f'{k}_wgt.csv')
    dfB = dfB.loc[dfB.session_type.str.contains(USE2)]
    dfB['user'] = dfB.session_type.str.partition('_')[0].astype('int32')
    dfB = dfB.drop('session_type',axis=1)

    tmpB = dfB.labels.str.split(' ').explode().astype('float32').rename('item').to_frame()
    tmpB['pos'] = np.tile( np.arange(50),tmpB.shape[0]//50 ).astype('int32')

    dfB = dfB[['user']]
    dfB = dfB.merge(tmpB, left_index=True, right_index=True)
    dfB.columns = ['user',f'score_{j+OFFSET}','pos']
    del tmpB

    dfA = dfA.merge(dfB, on=['user','pos'], how='left').drop('pos',axis=1)
    dfA = dfA.loc[dfA[f'score_{j+OFFSET}']!=-2]
    del dfB
      
    users = users.merge(dfA.to_pandas(),on=['user','item'],how='left').fillna(-3)
    del dfA
    
    gc.collect()
    
    print(k,users.shape)

In [ ]:
USE2 = 'clicks'
OFFSET = 23
for j,k in enumerate(['../../data/candidate_scores/submission_v614']):

    dfA = cudf.read_csv(f'{k}.csv')
    dfA = dfA.loc[dfA.session_type.str.contains(USE2)]
    dfA['user'] = dfA.session_type.str.partition('_')[0].astype('int32')
    dfA = dfA.drop('session_type',axis=1)

    tmpA = dfA.labels.str.split(' ').explode().astype('int32').rename('item').to_frame()
    tmpA['pos'] = np.tile( np.arange(50),tmpA.shape[0]//50 ).astype('int32')

    dfA = dfA[['user']]
    dfA = dfA.merge(tmpA, left_index=True, right_index=True)
    del tmpA

    dfB = cudf.read_csv(f'{k}_wgt.csv')
    dfB = dfB.loc[dfB.session_type.str.contains(USE2)]
    dfB['user'] = dfB.session_type.str.partition('_')[0].astype('int32')
    dfB = dfB.drop('session_type',axis=1)

    tmpB = dfB.labels.str.split(' ').explode().astype('float32').rename('item').to_frame()
    tmpB['pos'] = np.tile( np.arange(50),tmpB.shape[0]//50 ).astype('int32')

    dfB = dfB[['user']]
    dfB = dfB.merge(tmpB, left_index=True, right_index=True)
    dfB.columns = ['user',f'score_{j+OFFSET}','pos']
    del tmpB

    dfA = dfA.merge(dfB, on=['user','pos'], how='left').drop('pos',axis=1)
    dfA = dfA.loc[dfA[f'score_{j+OFFSET}']!=-2]
    del dfB
      
    users = users.merge(dfA.to_pandas(),on=['user','item'],how='left').fillna(-3)
    del dfA
    
    gc.collect()
    
    print(k,users.shape)

In [ ]:
#USE2 = 'orders'
OFFSET = 24

for j,(k,USE2) in enumerate( zip( range(31,34),['orders','orders','clicks'] ) ):

    dfA = cudf.read_csv(f'../../data/candidate_scores/submission_v564_v{k}.csv')
    dfA = dfA.loc[dfA.session_type.str.contains(USE2)]
    dfA['user'] = dfA.session_type.str.partition('_')[0].astype('int32')
    dfA = dfA.drop('session_type',axis=1)

    tmpA = dfA.labels.str.split(' ').explode().astype('int32').rename('item').to_frame()
    tmpA['pos'] = np.tile( np.arange(50),tmpA.shape[0]//50 ).astype('int32')

    dfA = dfA[['user']]
    dfA = dfA.merge(tmpA, left_index=True, right_index=True)
    del tmpA

    dfB = cudf.read_csv(f'../../data/candidate_scores/submission_v564_wgt_v{k}.csv')
    dfB = dfB.loc[dfB.session_type.str.contains(USE2)]
    dfB['user'] = dfB.session_type.str.partition('_')[0].astype('int32')
    dfB = dfB.drop('session_type',axis=1)

    tmpB = dfB.labels.str.split(' ').explode().astype('float32').rename('item').to_frame()
    tmpB['pos'] = np.tile( np.arange(50),tmpB.shape[0]//50 ).astype('int32')

    dfB = dfB[['user']]
    dfB = dfB.merge(tmpB, left_index=True, right_index=True)
    dfB.columns = ['user',f'score_{j+OFFSET}','pos']
    del tmpB

    dfA = dfA.merge(dfB, on=['user','pos'], how='left').drop('pos',axis=1)
    dfA = dfA.loc[dfA[f'score_{j+OFFSET}']!=-2]
    del dfB
      
    #dfA = dfA.rename({'pos':f'pos_{k}'},axis=1)
    users = users.merge(dfA.to_pandas(),on=['user','item'],how='left').fillna(-3)
    del dfA
    
    gc.collect()
    
    print(k,users.shape)

In [ ]:
#users = users.to_pandas()
#gc.collect()

In [ ]:
users.head()

# User and Item Features

In [ ]:
item_features = pd.read_parquet('../../data/item_user_features/item12.pqt')
for c in item_features.columns:
    if str(item_features[c].dtype)=='int32':
        print(c, item_features[c].dtype)
        item_features[c] = item_features[c].astype('float32')
print( item_features.shape )
item_features.head()

In [ ]:
users = users.merge(item_features, left_on='item', right_index=True, how='left')
del item_features
gc.collect()
users.head()

In [ ]:
item_features = pd.read_parquet('../../data/item_user_features/item13.pqt')
for c in item_features.columns:
    if str(item_features[c].dtype)=='int32':
        print(c, item_features[c].dtype)
        item_features[c] = item_features[c].astype('float32')
print( item_features.shape )
item_features.head()

In [ ]:
users = users.merge(item_features, left_on='item', right_index=True, how='left')
del item_features
gc.collect()
users.head()

In [ ]:
item_features = pd.read_parquet('../../data/item_user_features/item4.pqt')
for c in item_features.columns:
    if str(item_features[c].dtype)=='int32':
        print(c, item_features[c].dtype)
        item_features[c] = item_features[c].astype('float32')
print( item_features.shape )
item_features.head()

In [ ]:
#users = users.to_pandas()

In [ ]:
users = users.merge(item_features, left_on='item', right_index=True, how='left')

In [ ]:
users.head()

In [ ]:
del item_features
gc.collect()

In [ ]:
item_features = pd.read_parquet('../../data/item_user_features/item10.pqt')
for c in item_features.columns:
    if str(item_features[c].dtype)=='int32':
        print(c, item_features[c].dtype)
        item_features[c] = item_features[c].astype('float32')
print( item_features.shape )
users = users.merge(item_features, left_on='item', right_index=True, how='left')
users.head()

In [ ]:
del item_features
gc.collect()

In [ ]:
user_features = pd.read_parquet('../../data/item_user_features/user4.pqt')
for c in user_features.columns:
    if str(user_features[c].dtype)=='int32':
        print(c, user_features[c].dtype)
        user_features[c] = user_features[c].astype('float32')
print( user_features.shape )
user_features.head()

In [ ]:
users = users.merge(user_features, left_on='user', right_index=True, how='left')

In [ ]:
del user_features
gc.collect()

In [ ]:
users.head()

In [ ]:
for c in users.columns:
    print(c,users[c].dtype)

In [ ]:
for c in users.columns:
    t = str(users[c].dtype)
    if (t=='int64'): 
        users[c] = users[c].astype('int32')
        print('convert',c,'to int32')
    if (t=='float64')|(t=='Int32')|(t=='Float32'):
        users[c] = users[c].astype('float32')
        print('convert',c,'to float32')

In [ ]:
#%%time
#users = users.sort_values(['user','pos']).reset_index(drop=True)

In [ ]:
#users = users.reset_index(drop=True)
#users = users.to_pandas()
#gc.collect()

In [ ]:
users.head()

In [ ]:
users.to_parquet(f'../../data/train_with_features/train_v{VER}_{PART}.pqt',index=False)